## 0. Preparation

### Importing packages

In [1]:
import os
import glob
import pandas as pd
import numpy as np
import requests
import random
import time
import winsound
import json
import re
from datetime import datetime
from scrapy import Selector

### Defining Constants & Functions

In [2]:
DF_LTBX_DIR = 'C:\\Users\\willi\\Documents\\Data Science\\Projeto Críticas\\Letterboxd\\Most Popular Films CSVs\\2023\\STAGE'

In [3]:
def scrape_letterboxd_popularity_2(MAX_PAGES, SEARCH_MODE):
    
    #This script replaces 'scrape_letterboxd_popularity'
    
    #Input Variables:
    # - MAX_PAGES: How deep will the script run. Each page contains 72 film entries.
    # - SEARCH_MODE: String defining how data is sorted (by month, week, year or all-time)
    
    #Output Variable:
    # - FILM_DF: Dataframe containing movie data.
    # Note: 2022-03-21: Release Year and Poster URL info are not being retrieved.
    
    #Initializing variables
    page_index = 1
    last_url = ""
    next_url = "begin"
    pop_index = 0
    URL_LIST = []
    SEARCH_RANGE = { "MONTH":"this/month" , "WEEK":"this/week", "YEAR":"this/year", "ALL":"" }
    URL_BASE_STRING = "https://letterboxd.com/films/ajax/popular/{0}/size/small/page/{1}/".format(SEARCH_RANGE[SEARCH_MODE],"{0}")
    
    #Main Loop    
    while (page_index <= MAX_PAGES):
        print("Reading Page {}...".format(page_index))
        url = URL_BASE_STRING.format(page_index)
        req = requests.get(url)
        html = req.text
        html_selector = Selector(text=html)

        #Extract Data From Landing Page HTML
        #post_urls = html_selector.xpath('//div[contains(@class,"criticas-list")]//a/@href').extract()
        all_films = html_selector.xpath('//div[contains(@class,"linked-film-poster")]/@data-film-slug').extract()
        all_ids = html_selector.xpath('//div[contains(@class,"linked-film-poster")]/@data-film-id').extract()
        all_titles = html_selector.xpath('//img[contains(@class,"image")]/@alt').extract()
        #print(all_titles)
        #all_urls = html_selector.xpath('//a/@href').extract()
        #all_ids = html_selector.xpath('//div/@data-film-id').extract()
        #all_titles = html_selector.xpath('//div/@data-film-name').extract()
        #all_posters = html_selector.xpath('//div/@data-poster-url').extract()
        #all_release_years = html_selector.xpath('//div/@data-film-release-year').extract()

        for row_index in range(len(all_ids)):
            pop_index = pop_index + 1
            URL_ROW = [all_titles[row_index]] #get TITLE
            URL_ROW.append(0) #no RELEASE_YEAR info
            URL_ROW.append(all_films[row_index].split('/')[2]) #get SHORT_URL
            URL_ROW.append(pop_index) #save popularity index POP_INDEX
            URL_ROW.append(all_ids[row_index]) #get ID
            URL_ROW.append("") #no POSTER_URL info
            URL_LIST.append(URL_ROW) #list of lists to be turned into data frame

        #last_url = "/films/popular/{0}/size/small/page/{1}/".format(SEARCH_RANGE, page_index-1)
        #next_url = all_urls[-1]
        page_index = page_index + 1

        if MAX_PAGES > 1:        
            waittime = 3 + random.randint(1,5)
            print("... waiting {} seconds ...".format(waittime))
            time.sleep(waittime)

    df_columns = ['TITLE','RELEASE_YEAR','SHORT_URL', 'POP_INDEX','ID','POSTER_URL']
    FILM_DF = pd.DataFrame(URL_LIST, columns=df_columns)
    FILM_DF['DATE'] = datetime.today()
    FILM_DF['MODE'] = SEARCH_MODE
    
    return FILM_DF

In [4]:
#scrape_letterboxd_popularity_2(1, "WEEK")

In [5]:
def count_fans(fans_string):
    
    if pd.isna(fans_string): return 0
    
    fans_string = fans_string.lower()
    
    if len(fans_string) == 0:
        fan_count = 0
    elif fans_string.find('k') != -1:
        fans_string = fans_string.split('k')[0]
        fan_count = float(fans_string)
        fan_count = fan_count * 1000
    else:
        fans_string = fans_string.split()[0]
        fan_count = float(fans_string)
    return int(fan_count)

def get_ratings_from_title(ltbxd_title):   
    
    #Input Variable: A string representing the film. E.g.: "juno"
    #Output Variable: A dataframe containing the following data:
    # - Number of fans (text)
    # - List of ratings (# per rate)
    # - Sum of ratings (total #)
    # - Average rating
    # - Discord index
    
    url = 'https://letterboxd.com/csi/film/{}/rating-histogram/'.format(ltbxd_title)
    
    req = requests.get(url)
    html = req.text
    html_selector = Selector(text=html)
    
    RAT_ROW = { "SHORT_URL":'', "FANS":'', "RATINGS":[], "SUM": 0, "MEAN": 0, "DISCORD":0, "DATE":datetime.today() }

    RAT_ROW['SHORT_URL'] = ltbxd_title
    fans = html_selector.xpath('//a[contains(@class,"all-link")]/text()').extract()
    #RAT_ROW.at[0,'FANS']= fans
    if len(fans) == 0:
        RAT_ROW['FANS'] = ''
    else:
        RAT_ROW['FANS'] = fans[0]
    
    test_summary = html_selector.xpath('//a[contains(@title,"Not enough ratings to calculate average")]/text()').extract()
    
    #if test_summary==[]:
        #print("Test summary returned nothing")
        #RAT_ROW.at[0,'SUMMARY'] = html_selector.xpath('//span[contains(@class,"average-rating")]/a/@title').extract()
    #else:
        #print("Test Summary returned something")
        #RAT_ROW.at[0,'SUMMARY'] = "Not enough ratings to calculate average"
        
    ratings=[]
    ratings = html_selector.xpath('//a[contains(@class,"ir tooltip")]/@title').extract()
    #notas = [ int(ratings[index].split("\xa0")[0].replace(',','')) for index in range(0,len(ratings)) ]
    notas=[0,0,0,0,0,0,0,0,0,0]
    #print("Length Ratings")
    #print(len(ratings))
    #print(ratings)
    #print(url)

    if ratings != []:
        for index in range(0,len(ratings)):
            if '\xa0half-★ ' in ratings[index]: notas[0] = int(ratings[index].split("\xa0")[0].replace(',',''))
            if '\xa0★ ' in ratings[index]: notas[1] = int(ratings[index].split("\xa0")[0].replace(',',''))
            if '\xa0★½ ' in ratings[index]: notas[2] = int(ratings[index].split("\xa0")[0].replace(',',''))
            if '\xa0★★ ' in ratings[index]: notas[3] = int(ratings[index].split("\xa0")[0].replace(',',''))
            if '\xa0★★½ ' in ratings[index]: notas[4] = int(ratings[index].split("\xa0")[0].replace(',',''))
            if '\xa0★★★ ' in ratings[index]: notas[5] = int(ratings[index].split("\xa0")[0].replace(',',''))
            if '\xa0★★★½ ' in ratings[index]: notas[6] = int(ratings[index].split("\xa0")[0].replace(',',''))
            if '\xa0★★★★ ' in ratings[index]: notas[7] = int(ratings[index].split("\xa0")[0].replace(',',''))
            if '\xa0★★★★½ ' in ratings[index]: notas[8] = int(ratings[index].split("\xa0")[0].replace(',',''))
            if '\xa0★★★★★ ' in ratings[index]: notas[9] = int(ratings[index].split("\xa0")[0].replace(',',''))
        
        #print("Notas:")
        #print(notas)
        #print(RAT_ROW.SUMMARY)
        #RAT_ROW.at[0,'RATINGS'] = notas
        RAT_ROW['RATINGS'] = notas
        RAT_ROW['SUM'] = sum(notas);
        
        soma = 0
        for i in range(10):
            soma = soma + notas[i]*(0.5*(i+1))
        media = soma/RAT_ROW['SUM']
        RAT_ROW['MEAN'] = media
        #print("passou")
        
        array_total = []
        for i in range(10):
            counter = notas[i]
            array_total = np.append(array_total, np.full_like(np.arange(counter, dtype=int), 0.5*(i+1), dtype=np.double))
        RAT_ROW['DISCORD'] = np.std(array_total)/2.25
    else:
        #print("Filme Futuro")
        #RAT_ROW.at[0,'RATINGS'] = ""
        RAT_ROW['RATINGS'] = []
    
    return RAT_ROW

In [6]:
#get_ratings_from_title("barbie")

In [7]:
def scrape_letterboxd_ratings(FILM_LIST):  
    
    #Extracts film ratings from a list of strings
    
    FILM_RATINGS_DF = pd.DataFrame()
    counter = len(FILM_LIST)
    for film_title in FILM_LIST:
        RATINGS = get_ratings_from_title(film_title)
        print(RATINGS)
        FILM_RATINGS_DF = FILM_RATINGS_DF.append(pd.DataFrame.from_dict(RATINGS, orient='index').T).reset_index(drop = True)
        waittime = 3 + random.randint(1,3)
        counter = counter - 1
        print("... waiting {0} seconds ... {1} films pending".format(waittime,counter))
        time.sleep(waittime)
    return FILM_RATINGS_DF

def get_metadata_from_title(ltbxd_title):   
    
    url = 'https://letterboxd.com/film/{}/'.format(ltbxd_title)
    
    req = requests.get(url)
    html = req.text
    html_selector = Selector(text=html)
    
    film_dict = {"ID": '', 
                 "SHORT_URL": ltbxd_title, 
                 "TITLE": '',
                 "RELEASE_YEAR": 0,
                 "DISPLAY_TITLE": '',
                 "MAIN_IMAGE":'',
                 "POSTER_IMAGE": '',
                 "DIRECTORS_LIST":[], 
                 "STUDIOS_LIST":[], 
                 "ACTORS_LIST":[], 
                 "GENRE_LIST":[], 
                 "COUNTRIES_LIST":[],
                 "LANGUAGES_LIST": [],
                 "TAGLINE": '',
                 "DESCRIPTION": '',
                 "DURATION": 0,
                 "IMDB_LINK": '',
                 "TMDB_LINK": '',
                 "DATE":datetime.today() }

    film_dict['ID'] = html.split("filmData = { id: ")[1].split(",")[0]

    metadata = html_selector.xpath('//script[@type="application/ld+json"]/text()').extract()
    if len(metadata)==0:
        print("Não encontrou metadata json string - registro zerado")
        return film_dict
    
    metadata = metadata[0].split("\n/* <![CDATA[ */\n")[1].split("\n/* ]]> */\n")[0]
    metadata = json.loads(metadata)
    
    if 'name' in metadata.keys(): film_dict['TITLE'] = metadata['name']
    if 'releasedEvent' in metadata.keys() : film_dict['RELEASE_YEAR'] = int(metadata['releasedEvent'][0]['startDate'])
    if 'image' in metadata.keys(): film_dict['POSTER_IMAGE'] = metadata['image']
    if 'director' in metadata.keys(): film_dict['DIRECTORS_LIST'] = metadata['director']
    if 'productionCompany' in metadata.keys(): film_dict['STUDIOS_LIST'] = metadata['productionCompany']
    if 'actors' in metadata.keys(): film_dict['ACTORS_LIST'] = metadata['actors']
    if 'genre' in metadata.keys(): film_dict['GENRE_LIST'] = metadata['genre']
    if 'countryOfOrigin' in metadata.keys(): film_dict['COUNTRIES_LIST']  = metadata['countryOfOrigin']
    
    languages = html_selector.xpath('//a[contains(@href,"/films/language/")]/text()').extract()
    tagline = html_selector.xpath('//h4[@class="tagline"]/text()').extract()
    display_title = html_selector.xpath('//meta[@property="og:title"]/@content').extract()
    description = html_selector.xpath('//meta[@name="description"]/@content').extract()
    main_image = html_selector.xpath('//meta[@property="og:image"]/@content').extract()
    duration = html_selector.xpath('//p[@class="text-link text-footer"]/text()').extract()
    imdb_link = html_selector.xpath('//a[contains(@href,"www.imdb.com")]/@href').extract()
    tmdb_link = html_selector.xpath('//a[contains(@href,"www.themoviedb.org")]/@href').extract()
        
    film_dict['LANGUAGES_LIST'] = languages
    if (len(tagline) > 0): film_dict['TAGLINE']  = tagline[0]
    if (len(display_title) > 0): film_dict['DISPLAY_TITLE']  = display_title[0]   
    if (len(description) > 0): film_dict['DESCRIPTION']  = description[0]    
    if (len(main_image) > 0): film_dict['MAIN_IMAGE']  = main_image[0]   
    if (len(duration) > 0): film_dict['DURATION']  = [int(s) for s in duration[0].split() if s.isdigit()]
    if (len(film_dict['DURATION']) > 0): 
        film_dict['DURATION'] = film_dict['DURATION'][0]
    else:
        film_dict['DURATION'] = 0
    if (len(description) > 0) & (len(imdb_link) > 0) : film_dict['IMDB_LINK']  = imdb_link[0]   
    if (len(description) > 0): film_dict['TMDB_LINK']  = tmdb_link[0] 

    return film_dict

In [8]:
#get_metadata_from_title("juno")

In [9]:
def scrape_letterboxd_metadata(FILM_LIST):  
    FILM_METADATA_DF = pd.DataFrame()
    counter = len(FILM_LIST)
    for film_title in FILM_LIST:
        print("Reading: {}".format(film_title))
        METADATA = get_metadata_from_title(film_title)
        FILM_METADATA_DF = FILM_METADATA_DF.append(pd.DataFrame.from_dict(METADATA, orient='index').T).reset_index(drop = True)
        waittime = 5 + random.randint(1,5)
        counter = counter - 1
        print("... waiting {0} seconds ... {1} films pending".format(waittime,counter))
        time.sleep(waittime)
    print("DONE")
    return FILM_METADATA_DF

In [10]:
#scrape_letterboxd_metadata(["fear-street-1994"])

In [11]:
def scrape_letterboxd_numbers(FILM_LIST):  
    
    #Extracts film ratings from a list of strings
    
    FILM_NUMBERS_DF = pd.DataFrame()
    counter = len(FILM_LIST)
    for film_title in FILM_LIST:
        NUMBERS = get_numbers_from_title(film_title)
        print(NUMBERS)
        FILM_NUMBERS_DF = FILM_NUMBERS_DF.append(pd.DataFrame.from_dict(NUMBERS, orient='index').T).reset_index(drop = True)
        waittime = 2 + random.randint(1,3)
        counter = counter - 1
        print("... waiting {0} seconds ... {1} films pending".format(waittime,counter))
        time.sleep(waittime)
    return FILM_NUMBERS_DF

def get_numbers_from_title(ltbxd_title):   
    
    url = 'https://letterboxd.com/film/{}/members/'.format(ltbxd_title)
    
    req = requests.get(url)
    html = req.text
    html_selector = Selector(text=html)
    
    film_dict = {"SHORT_URL": ltbxd_title, 
                 "TITLE": '',
                 "RELEASE_YEAR": 0,
                 "TOTAL_AUDIENCE": 0,
                 "FANS": 0,
                 "LIKES": 0,
                 "REVIEWS": 0,
                 "LISTS": 0,
                 "DATE":datetime.today() }
    
    film_xpath = '//a[contains(@href,"/film/' + ltbxd_title + '")]/text()'
    film_title = html_selector.xpath(film_xpath).extract()
    
    release_year = html_selector.xpath('//a[contains(@href,"/films/year/")]/text()').extract()
    
    audience_xpath = '//a[contains(@href,"/film/' + ltbxd_title + '/members/")]/@title'
    total_audience = html_selector.xpath(audience_xpath).extract()
    
    fans_xpath = '//a[contains(@href,"/film/' + ltbxd_title + '/fans/")]/@title'
    fans = html_selector.xpath(fans_xpath).extract()
    
    likes_xpath = '//a[contains(@href,"/film/' + ltbxd_title + '/likes/")]/@title'
    likes = html_selector.xpath(likes_xpath).extract()
    
    reviews_xpath = '//a[contains(@href,"/film/' + ltbxd_title + '/reviews/")]/@title'
    reviews = html_selector.xpath(reviews_xpath).extract()
    
    lists_xpath = '//a[contains(@href,"/film/' + ltbxd_title + '/lists/")]/@title'
    lists = html_selector.xpath(lists_xpath).extract()
    
    if (len(film_title) > 0): film_dict['TITLE']  = film_title[0]
    if (len(release_year) > 0): film_dict['RELEASE_YEAR']  = int(release_year[0])
    if (len(total_audience) > 0): film_dict['TOTAL_AUDIENCE']  = int(''.join(re.findall('[0-9]+', total_audience[0])))
    if (len(fans) > 0): film_dict['FANS']  = int(''.join(re.findall('[0-9]+', fans[0])))
    if (len(likes) > 0): film_dict['LIKES']  = int(''.join(re.findall('[0-9]+', likes[0])))
    if (len(reviews) > 0): film_dict['REVIEWS']  = int(''.join(re.findall('[0-9]+', reviews[0])))
    if (len(lists) > 0): film_dict['LISTS']  = int(''.join(re.findall('[0-9]+', lists[0])))

    return film_dict

In [12]:
#scrape_letterboxd_numbers(["fellini-satyricon"])

## 1. Most Popular Films (Week, Month, Year & All Time)

### 1.1 Scraping Today's 'Most Popular' Data

In [13]:
MAX_PAGES = 14 #Each page lists 72 films; Choose 3 for 200 films; Choose 14 for 1000 films; Choose 70 for 5000 films
SEARCH_MODES = ["WEEK", "MONTH", "YEAR", "ALL"] #It can be "WEEK", "MONTH", "YEAR" or "ALL"

FILM_DF_TODAY = pd.DataFrame()
for mode in SEARCH_MODES:
    FILM_DF = scrape_letterboxd_popularity_2(MAX_PAGES, mode)
    #print(FILM_DF)
    FILM_DF_TODAY = FILM_DF.append(FILM_DF_TODAY).reset_index(drop = True)
print("OK")

Reading Page 1...
... waiting 6 seconds ...
Reading Page 2...
... waiting 6 seconds ...
Reading Page 3...
... waiting 5 seconds ...
Reading Page 4...
... waiting 8 seconds ...
Reading Page 5...
... waiting 5 seconds ...
Reading Page 6...
... waiting 8 seconds ...
Reading Page 7...
... waiting 7 seconds ...
Reading Page 8...
... waiting 7 seconds ...
Reading Page 9...
... waiting 8 seconds ...
Reading Page 10...
... waiting 4 seconds ...
Reading Page 11...
... waiting 8 seconds ...
Reading Page 12...
... waiting 4 seconds ...
Reading Page 13...
... waiting 4 seconds ...
Reading Page 14...
... waiting 8 seconds ...
Reading Page 1...
... waiting 6 seconds ...
Reading Page 2...
... waiting 8 seconds ...
Reading Page 3...
... waiting 4 seconds ...
Reading Page 4...
... waiting 7 seconds ...
Reading Page 5...
... waiting 6 seconds ...
Reading Page 6...
... waiting 7 seconds ...
Reading Page 7...
... waiting 4 seconds ...
Reading Page 8...
... waiting 8 seconds ...
Reading Page 9...
... waiti

In [14]:
len(FILM_DF_TODAY)

4032

#### Checking for errors before saving CSV file

In [15]:
# Only do this step if you need to inspect previous files
# filename = DF_LTBX_DIR + '\\LTBX_FILMS_POP_20221225.csv'
# FILM_DF_TODAY = pd.read_csv(filename).drop('Unnamed: 0',axis=1)

In [16]:
# Are there any empty titles?
FILM_DF_TODAY[FILM_DF_TODAY.TITLE == '']

,TITLE,RELEASE_YEAR,SHORT_URL,POP_INDEX,ID,POSTER_URL,DATE,MODE


In [17]:
# Are there any empty cells (TITLE, SHORT_URL, ID)?
print(FILM_DF_TODAY[['TITLE','SHORT_URL', 'ID']].isnull().values.any())
print(FILM_DF_TODAY[FILM_DF_TODAY['TITLE'].isnull()])
print(FILM_DF_TODAY[FILM_DF_TODAY['SHORT_URL'].isnull()])
print(FILM_DF_TODAY[FILM_DF_TODAY['ID'].isnull()])

False
Empty DataFrame
Columns: [TITLE, RELEASE_YEAR, SHORT_URL, POP_INDEX, ID, POSTER_URL, DATE, MODE]
Index: []
Empty DataFrame
Columns: [TITLE, RELEASE_YEAR, SHORT_URL, POP_INDEX, ID, POSTER_URL, DATE, MODE]
Index: []
Empty DataFrame
Columns: [TITLE, RELEASE_YEAR, SHORT_URL, POP_INDEX, ID, POSTER_URL, DATE, MODE]
Index: []


In [18]:
print("Is the row count not expected?")
n_rows = len(FILM_DF_TODAY)
print(n_rows != MAX_PAGES * len(SEARCH_MODES) * 72)
print()
print("Are there film duplicates (same ID)?")
print(len(FILM_DF_TODAY[FILM_DF_TODAY.MODE == 'ALL']['ID'].unique()) != n_rows/4)
print(len(FILM_DF_TODAY[FILM_DF_TODAY.MODE == 'YEAR']['ID'].unique()) != n_rows/4)
print(len(FILM_DF_TODAY[FILM_DF_TODAY.MODE == 'WEEK']['ID'].unique()) != n_rows/4)
print(len(FILM_DF_TODAY[FILM_DF_TODAY.MODE == 'MONTH']['ID'].unique()) != n_rows/4)
print()
print("Are there film duplicates (same short url)?")
#Are there any film ID with more than one short_url? If yes, Letterboxd possibly changed the short_url.
GROUPED_DF = FILM_DF_TODAY[['SHORT_URL','ID','MODE']].groupby(['ID','MODE']).count().sort_values('SHORT_URL', ascending=False).reset_index()
print(GROUPED_DF[GROUPED_DF.SHORT_URL != 1].values.any() != False)

Is the row count not expected?
False

Are there film duplicates (same ID)?
False
False
False
False

Are there film duplicates (same short url)?
False


In [19]:
len(GROUPED_DF[GROUPED_DF.SHORT_URL > 1])

0

In [20]:
GROUPED_DF[GROUPED_DF.SHORT_URL > 1]

,ID,MODE,SHORT_URL


In [21]:
len(FILM_DF_TODAY[FILM_DF_TODAY.MODE == 'WEEK']['ID'].unique())

1008

In [22]:
FILM_DF_TODAY[FILM_DF_TODAY.MODE == 'ALL'].groupby('ID').count().sort_values(by = 'MODE')

,TITLE,RELEASE_YEAR,SHORT_URL,POP_INDEX,POSTER_URL,DATE,MODE
ID,,,,,,,
101990,1,1,1,1,1,1,1
51366,1,1,1,1,1,1,1
51379,1,1,1,1,1,1,1
51380,1,1,1,1,1,1,1
51381,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...
422682,1,1,1,1,1,1,1
424348,1,1,1,1,1,1,1
42531,1,1,1,1,1,1,1


In [23]:
FILM_DF_TODAY[FILM_DF_TODAY['ID'] == '666269']

,TITLE,RELEASE_YEAR,SHORT_URL,POP_INDEX,ID,POSTER_URL,DATE,MODE
831,White Noise,0,white-noise-2022,832,666269,,2023-03-09 23:35:56.458666,ALL
1047,White Noise,0,white-noise-2022,40,666269,,2023-03-09 23:34:29.586170,YEAR
2405,White Noise,0,white-noise-2022,390,666269,,2023-03-09 23:33:00.052487,MONTH
3594,White Noise,0,white-noise-2022,571,666269,,2023-03-09 23:31:23.944988,WEEK


#### Fixing Errors, If Any

In [24]:
FILM_DF_TODAY.loc[FILM_DF_TODAY['ID'] == '913163','TITLE'] = '“Sr.”'

In [25]:
FILM_DF_TODAY[FILM_DF_TODAY['ID'] == '913163']

,TITLE,RELEASE_YEAR,SHORT_URL,POP_INDEX,ID,POSTER_URL,DATE,MODE


#### Saving CSV file

In [26]:
FILM_DF_TODAY.to_csv(DF_LTBX_DIR + '\\LTBX_FILMS_POP_' + datetime.today().strftime("%Y%m%d") + '.csv')

### 1.2 Consolidating Daily 'Most Popular' Files

In [27]:
filenames = glob.glob(DF_LTBX_DIR + "\\LTBX_FILMS_POP_*.csv")
dfs = []
file_index = 0
for filename in filenames:
    new_pd = pd.read_csv(filename, parse_dates=['DATE'])
    new_pd['FILE_INDEX'] = file_index
    file_index = file_index + 1
    dfs.append(new_pd)
# Concatenate all data into one DataFrame
LTBX_FILMS_POP_DF = pd.concat(dfs, ignore_index=True)
# Keeping only the useful columns
LTBX_FILMS_POP_DF = LTBX_FILMS_POP_DF[['TITLE', 'SHORT_URL', 'POP_INDEX', 'ID', 'DATE', 'MODE', 'FILE_INDEX']]

#LTBX_FILMS_POP_DF[LTBX_FILMS_POP_DF.SHORT_URL == 'emily-the-criminal'].sort_values(by = ['MODE','DATE'])

In [28]:
FILM_LIST = LTBX_FILMS_POP_DF.SHORT_URL.unique()
len(FILM_LIST) #Tem que ser 1366

1637

In [29]:
for mode_type in ['ALL', 'MONTH', 'WEEK', 'YEAR']:
    for short_url in FILM_LIST:
        results_index = LTBX_FILMS_POP_DF[(LTBX_FILMS_POP_DF.SHORT_URL == short_url) & (LTBX_FILMS_POP_DF.MODE == mode_type)].index
        results_len = len(results_index)
        if results_len > 0:
            previous_position = LTBX_FILMS_POP_DF.loc[results_index[0]].POP_INDEX
            previous_appearance = LTBX_FILMS_POP_DF.loc[results_index[0]].FILE_INDEX
            is_first_appearance = True
            for index in results_index:
                if is_first_appearance: #All POP_DIFF will be 0
                    LTBX_FILMS_POP_DF.loc[index, 'POP_DIFF'] = 'NEW'
                    is_first_appearance = False
                else: #Following entries being read
                    current_appearance = LTBX_FILMS_POP_DF.loc[index].FILE_INDEX
                    current_position = LTBX_FILMS_POP_DF.loc[index].POP_INDEX
                    days_since_last_appearance = current_appearance - previous_appearance
                    #if delta_appearance > 0: print(LTBX_FILMS_POP_DF.loc[index])
                    if days_since_last_appearance == 1: #The movie was in the list the day before
                        LTBX_FILMS_POP_DF.loc[index, 'POP_DIFF'] = previous_position - current_position
                    else: #Filme saiu da lista e reentrou
                        LTBX_FILMS_POP_DF.loc[index, 'POP_DIFF'] = 'OLD'
                        #LTBX_FILMS_POP_DF.loc[index, 'POP_DIFF'] = delta_appearance
                    previous_appearance = current_appearance
                    previous_position = current_position   

In [30]:
LTBX_FILMS_POP_DF.to_csv(DF_LTBX_DIR + '\\LTBX_ALL_FILMS_POP_FROM_20221225_TO_' + datetime.today().strftime("%Y%m%d") + '.csv')

## 2. Film Ratings

### 2.1 Scraping Most Current Ratings Data
#### Retrieves ratings data from a cumulative film list

In [31]:
len(FILM_LIST)

1637

In [32]:
NEW_FILM_RATINGS_DF = scrape_letterboxd_ratings(FILM_LIST[:])

{'SHORT_URL': 'parasite-2019', 'FANS': '60K fans', 'RATINGS': [1667, 3045, 1406, 8489, 7431, 51217, 62385, 334556, 303704, 988267], 'SUM': 1762167, 'MEAN': 4.573782450811983, 'DISCORD': 0.271615316949543, 'DATE': datetime.datetime(2023, 3, 10, 0, 19, 8, 738059)}
... waiting 4 seconds ... 1636 films pending
{'SHORT_URL': 'joker-2019', 'FANS': '15K fans', 'RATINGS': [7722, 18484, 12690, 61158, 53676, 214236, 198443, 503343, 193094, 370297], 'SUM': 1633143, 'MEAN': 3.8997959762249845, 'DISCORD': 0.40731704788788986, 'DATE': datetime.datetime(2023, 3, 10, 0, 19, 13, 90694)}
... waiting 5 seconds ... 1635 films pending
{'SHORT_URL': 'fight-club', 'FANS': '68K fans', 'RATINGS': [2102, 5401, 2734, 17889, 14921, 97341, 104740, 392255, 235588, 545626], 'SUM': 1418597, 'MEAN': 4.299700690188969, 'DISCORD': 0.33317525559438443, 'DATE': datetime.datetime(2023, 3, 10, 0, 19, 18, 360381)}
... waiting 6 seconds ... 1634 films pending
{'SHORT_URL': 'the-batman', 'FANS': '30K fans', 'RATINGS': [3037, 8

{'SHORT_URL': 'inglourious-basterds', 'FANS': '31K fans', 'RATINGS': [1177, 3054, 1855, 11106, 10356, 70752, 85634, 322504, 208273, 389755], 'SUM': 1104466, 'MEAN': 4.293949745850031, 'DISCORD': 0.31640808686148114, 'DATE': datetime.datetime(2023, 3, 10, 0, 21, 30, 474249)}
... waiting 5 seconds ... 1610 films pending
{'SHORT_URL': 'avengers-endgame', 'FANS': '13K fans', 'RATINGS': [7035, 14360, 8562, 40741, 40177, 162127, 145503, 323159, 137884, 300946], 'SUM': 1180494, 'MEAN': 3.9188089901346386, 'DISCORD': 0.41884719502135875, 'DATE': datetime.datetime(2023, 3, 10, 0, 21, 35, 750056)}
... waiting 6 seconds ... 1609 films pending
{'SHORT_URL': 'the-truman-show', 'FANS': '28K fans', 'RATINGS': [701, 2517, 1449, 11242, 11246, 91382, 112939, 370042, 183166, 320694], 'SUM': 1105378, 'MEAN': 4.1912879576036435, 'DISCORD': 0.31527290708884625, 'DATE': datetime.datetime(2023, 3, 10, 0, 21, 42, 25240)}
... waiting 4 seconds ... 1608 films pending
{'SHORT_URL': 'eternal-sunshine-of-the-spotle

{'SHORT_URL': 'thor-ragnarok', 'FANS': '6.9K fans', 'RATINGS': [3536, 9039, 6135, 30306, 31697, 139724, 151949, 315652, 109705, 176908], 'SUM': 974651, 'MEAN': 3.849252706866355, 'DISCORD': 0.38078733895129147, 'DATE': datetime.datetime(2023, 3, 10, 0, 23, 49, 223933)}
... waiting 4 seconds ... 1583 films pending
{'SHORT_URL': 'spider-man-homecoming', 'FANS': '1.9K fans', 'RATINGS': [3290, 8628, 7342, 40140, 48827, 224955, 206356, 298089, 56121, 90163], 'SUM': 983911, 'MEAN': 3.573961466026907, 'DISCORD': 0.35744446030634264, 'DATE': datetime.datetime(2023, 3, 10, 0, 23, 53, 509243)}
... waiting 5 seconds ... 1582 films pending
{'SHORT_URL': 'shutter-island', 'FANS': '18K fans', 'RATINGS': [914, 3149, 2201, 15145, 15990, 89474, 113702, 302199, 149262, 219445], 'SUM': 911481, 'MEAN': 4.082642973358743, 'DISCORD': 0.3351416431057179, 'DATE': datetime.datetime(2023, 3, 10, 0, 23, 58, 790777)}
... waiting 6 seconds ... 1581 films pending
{'SHORT_URL': 'us-2019', 'FANS': '6.4K fans', 'RATIN

{'SHORT_URL': 'goodfellas', 'FANS': '35K fans', 'RATINGS': [462, 972, 623, 3945, 4331, 28393, 41735, 161794, 133013, 291351], 'SUM': 666619, 'MEAN': 4.43221090307957, 'DISCORD': 0.289443200405299, 'DATE': datetime.datetime(2023, 3, 10, 0, 26, 7, 731388)}
... waiting 4 seconds ... 1557 films pending
{'SHORT_URL': 'the-perks-of-being-a-wallflower', 'FANS': '38K fans', 'RATINGS': [2383, 6208, 4480, 22348, 22900, 101055, 101286, 210430, 81942, 202721], 'SUM': 755753, 'MEAN': 3.9666339399248165, 'DISCORD': 0.3969046609669198, 'DATE': datetime.datetime(2023, 3, 10, 0, 26, 11, 990354)}
... waiting 5 seconds ... 1556 films pending
{'SHORT_URL': 'nightcrawler', 'FANS': '9.1K fans', 'RATINGS': [663, 2089, 1451, 9566, 10911, 64866, 100166, 277564, 130146, 115531], 'SUM': 712953, 'MEAN': 4.0251650529558045, 'DISCORD': 0.3052593877686488, 'DATE': datetime.datetime(2023, 3, 10, 0, 26, 17, 282763)}
... waiting 6 seconds ... 1555 films pending
{'SHORT_URL': 'up', 'FANS': '3.7K fans', 'RATINGS': [822, 

{'SHORT_URL': 'spider-man', 'FANS': '4.4K fans', 'RATINGS': [1099, 4094, 2958, 20296, 23410, 135276, 149808, 251781, 62462, 97635], 'SUM': 748819, 'MEAN': 3.7588943389524037, 'DISCORD': 0.3426559790843367, 'DATE': datetime.datetime(2023, 3, 10, 0, 28, 21, 839766)}
... waiting 4 seconds ... 1531 films pending
{'SHORT_URL': 'blade-runner', 'FANS': '28K fans', 'RATINGS': [1191, 3750, 2742, 13930, 15086, 60157, 69301, 169756, 92680, 173455], 'SUM': 602048, 'MEAN': 4.086745575103646, 'DISCORD': 0.373294945426128, 'DATE': datetime.datetime(2023, 3, 10, 0, 28, 26, 103617)}
... waiting 6 seconds ... 1530 films pending
{'SHORT_URL': 'titanic-1997', 'FANS': '16K fans', 'RATINGS': [3643, 9361, 5581, 32641, 30129, 139851, 117257, 225872, 71941, 180177], 'SUM': 816453, 'MEAN': 3.8192259689167654, 'DISCORD': 0.4123392597470309, 'DATE': datetime.datetime(2023, 3, 10, 0, 28, 32, 362236)}
... waiting 6 seconds ... 1529 films pending
{'SHORT_URL': 'zodiac', 'FANS': '7.8K fans', 'RATINGS': [714, 2398, 17

...

{'SHORT_URL': 'the-last-airbender', 'FANS': '95 fans', 'RATINGS': [64806, 29377, 9046, 10344, 4196, 4825, 1539, 1416, 223, 1110], 'SUM': 126882, 'MEAN': 1.09240081335414, 'DISCORD': 0.39321861104586303, 'DATE': datetime.datetime(2023, 3, 10, 2, 31, 48, 968294)}
... waiting 6 seconds ... 135 films pending
{'SHORT_URL': 'venus-2022-1', 'FANS': '10 fans', 'RATINGS': [79, 172, 207, 568, 743, 1219, 772, 452, 93, 95], 'SUM': 4400, 'MEAN': 2.8581818181818184, 'DISCORD': 0.3990451822076268, 'DATE': datetime.datetime(2023, 3, 10, 2, 31, 55, 267567)}
... waiting 4 seconds ... 134 films pending
{'SHORT_URL': 'ruby-sparks', 'FANS': '1.1K fans', 'RATINGS': [301, 652, 652, 2705, 3904, 14053, 18280, 20562, 5384, 6700], 'SUM': 73193, 'MEAN': 3.5941346850108618, 'DISCORD': 0.35934151600991737, 'DATE': datetime.datetime(2023, 3, 10, 2, 31, 59, 535558)}
... waiting 4 seconds ... 133 films pending
{'SHORT_URL': 'tampopo', 'FANS': '2.7K fans', 'RATINGS': [61, 78, 85, 353, 597, 2737, 5823, 17327, 12961, 150

{'SHORT_URL': 'wild-at-heart', 'FANS': '1.3K fans', 'RATINGS': [245, 614, 663, 2723, 3642, 12588, 18100, 28451, 10981, 11217], 'SUM': 89224, 'MEAN': 3.773642741863176, 'DISCORD': 0.35914189749668807, 'DATE': datetime.datetime(2023, 3, 10, 2, 34, 15, 500406)}
... waiting 6 seconds ... 107 films pending
{'SHORT_URL': 'sharper', 'FANS': '28 fans', 'RATINGS': [58, 166, 269, 1153, 2281, 5094, 3953, 2375, 417, 457], 'SUM': 16223, 'MEAN': 3.1674782715897183, 'DISCORD': 0.33157579834050577, 'DATE': datetime.datetime(2023, 3, 10, 2, 34, 21, 758312)}
... waiting 6 seconds ... 106 films pending
{'SHORT_URL': 'the-son-2022', 'FANS': '6 fans', 'RATINGS': [284, 733, 835, 1778, 1600, 1842, 1165, 928, 256, 227], 'SUM': 9648, 'MEAN': 2.620854063018242, 'DISCORD': 0.4542876323235165, 'DATE': datetime.datetime(2023, 3, 10, 2, 34, 28, 6683)}
... waiting 4 seconds ... 105 films pending
{'SHORT_URL': 'no-strings-attached', 'FANS': '126 fans', 'RATINGS': [1602, 4424, 4779, 15628, 15340, 25427, 10025, 8369, 1

{'SHORT_URL': 'tetris', 'FANS': '', 'RATINGS': [], 'SUM': 0, 'MEAN': 0, 'DISCORD': 0, 'DATE': datetime.datetime(2023, 3, 10, 2, 36, 43, 5442)}
... waiting 4 seconds ... 79 films pending
{'SHORT_URL': 'the-beach', 'FANS': '422 fans', 'RATINGS': [508, 1694, 2138, 8375, 10557, 24770, 17192, 12930, 2472, 3265], 'SUM': 83901, 'MEAN': 3.1221141583532974, 'DISCORD': 0.377017190042069, 'DATE': datetime.datetime(2023, 3, 10, 2, 36, 47, 289464)}
... waiting 4 seconds ... 78 films pending
{'SHORT_URL': 'picnic-at-hanging-rock', 'FANS': '855 fans', 'RATINGS': [113, 319, 339, 1417, 2042, 6505, 9069, 14145, 6353, 6216], 'SUM': 46518, 'MEAN': 3.790532697020508, 'DISCORD': 0.36382199785310015, 'DATE': datetime.datetime(2023, 3, 10, 2, 36, 51, 558357)}
... waiting 5 seconds ... 77 films pending
{'SHORT_URL': 'j-hope-in-the-box', 'FANS': '16 fans', 'RATINGS': [11, 7, 2, 8, 9, 53, 56, 213, 83, 1355], 'SUM': 1797, 'MEAN': 4.679744017807457, 'DISCORD': 0.3101224775983462, 'DATE': datetime.datetime(2023, 3,

{'SHORT_URL': 'south-park-bigger-longer-uncut', 'FANS': '1K fans', 'RATINGS': [375, 887, 740, 3030, 3973, 15932, 19279, 30021, 9747, 12228], 'SUM': 96212, 'MEAN': 3.726510206627032, 'DISCORD': 0.3694470175216372, 'DATE': datetime.datetime(2023, 3, 10, 2, 39, 17, 533493)}
... waiting 4 seconds ... 51 films pending
{'SHORT_URL': 'swallowed-2022', 'FANS': '', 'RATINGS': [23, 63, 73, 264, 339, 524, 440, 258, 57, 55], 'SUM': 2096, 'MEAN': 2.9747137404580153, 'DISCORD': 0.3911734456943204, 'DATE': datetime.datetime(2023, 3, 10, 2, 39, 21, 802134)}
... waiting 5 seconds ... 50 films pending
{'SHORT_URL': 'like-an-afternoon-dream', 'FANS': '21 fans', 'RATINGS': [7, 11, 13, 46, 98, 399, 718, 1534, 626, 677], 'SUM': 4129, 'MEAN': 3.976507628965851, 'DISCORD': 0.3154639984737597, 'DATE': datetime.datetime(2023, 3, 10, 2, 39, 27, 66461)}
... waiting 6 seconds ... 49 films pending
{'SHORT_URL': 'tusk-2014', 'FANS': '301 fans', 'RATINGS': [6054, 8291, 6654, 13547, 11145, 16677, 8744, 7170, 1184, 270

{'SHORT_URL': 'demon-slayer-kimetsu-no-yaiba-to-the-swordsmith-village', 'FANS': '8 fans', 'RATINGS': [98, 176, 110, 325, 295, 807, 774, 858, 163, 295], 'SUM': 3901, 'MEAN': 3.2166111253524736, 'DISCORD': 0.4708315749762626, 'DATE': datetime.datetime(2023, 3, 10, 2, 41, 52, 200871)}
... waiting 6 seconds ... 22 films pending
{'SHORT_URL': 'how-to-blow-up-a-pipeline', 'FANS': '15 fans', 'RATINGS': [7, 12, 7, 42, 60, 161, 244, 446, 213, 144], 'SUM': 1336, 'MEAN': 3.7870508982035926, 'DISCORD': 0.36461471051016897, 'DATE': datetime.datetime(2023, 3, 10, 2, 41, 58, 452590)}
... waiting 5 seconds ... 21 films pending
{'SHORT_URL': 'sunshine-2007', 'FANS': '931 fans', 'RATINGS': [209, 570, 725, 3034, 4733, 16336, 23463, 28007, 8966, 6198], 'SUM': 92241, 'MEAN': 3.622640691232749, 'DISCORD': 0.3353423377429863, 'DATE': datetime.datetime(2023, 3, 10, 2, 42, 3, 821101)}
... waiting 6 seconds ... 20 films pending
{'SHORT_URL': 'chris-rock-selective-outrage', 'FANS': '7 fans', 'RATINGS': [98, 152

In [33]:
NEW_FILM_RATINGS_DF['N_FANS'] = NEW_FILM_RATINGS_DF.FANS.apply(count_fans)
NEW_FILM_RATINGS_DF['MEAN_IND'] =  NEW_FILM_RATINGS_DF['MEAN']*1000
NEW_FILM_RATINGS_DF['MEAN_IND'] =  NEW_FILM_RATINGS_DF['MEAN_IND'].apply(round)
NEW_FILM_RATINGS_DF['DISCORD_IND'] =  NEW_FILM_RATINGS_DF['DISCORD']*1000
NEW_FILM_RATINGS_DF['DISCORD_IND'] =  NEW_FILM_RATINGS_DF['DISCORD_IND'].apply(round)

In [34]:
NEW_FILM_RATINGS_DF

,SHORT_URL,FANS,RATINGS,SUM,MEAN,DISCORD,DATE,N_FANS,MEAN_IND,DISCORD_IND
0,parasite-2019,60K fans,"[1667, 3045, 1406, 8489, 7431, 51217, 62385, 3...",1762167,4.573782,0.271615,2023-03-10 00:19:08.738059,60000,4574,272
1,joker-2019,15K fans,"[7722, 18484, 12690, 61158, 53676, 214236, 198...",1633143,3.899796,0.407317,2023-03-10 00:19:13.090694,15000,3900,407
2,fight-club,68K fans,"[2102, 5401, 2734, 17889, 14921, 97341, 104740...",1418597,4.299701,0.333175,2023-03-10 00:19:18.360381,68000,4300,333
3,the-batman,30K fans,"[3037, 8365, 5321, 31284, 31084, 134966, 16339...",1367511,4.050224,0.359323,2023-03-10 00:19:24.683866,30000,4050,359
4,knives-out-2019,26K fans,"[1770, 5996, 3503, 25118, 25233, 156212, 18347...",1526139,4.063602,0.333078,2023-03-10 00:19:30.960149,26000,4064,333
...,...,...,...,...,...,...,...,...,...,...
1632,burlesque,948 fans,"[613, 1759, 1976, 5961, 6256, 14356, 9431, 108...",61594,3.293097,0.467404,2023-03-10 02:43:24.939098,948,3293,467
1633,catherine-called-birdy,62 fans,"[44, 123, 160, 547, 934, 3649, 4446, 4556, 904...",16259,3.511532,0.329538,2023-03-10 02:43:29.217258,62,3512,330
1634,65,7 fans,"[45, 161, 238, 509, 457, 579, 277, 140, 25, 45]",2476,2.546446,0.401453,2023-03-10 02:43:35.476526,7,2546,401
1635,ripd,11 fans,"[3579, 7829, 7064, 12268, 8497, 9052, 3251, 19...",54497,2.150219,0.429081,2023-03-10 02:43:41.740881,11,2150,429


In [35]:
NEW_FILM_RATINGS_DF.to_csv(DF_LTBX_DIR + '\\LTBX_FILM_RATINGS_' + datetime.today().strftime("%Y%m%d") + '.csv')

### 2.2 Consolidating Daily Ratings Files

In [36]:
filenames = glob.glob(DF_LTBX_DIR + "\\LTBX_FILM_RATINGS_*.csv")
dfs = []
file_index = 0
for filename in filenames:
    new_pd = pd.read_csv(filename, parse_dates=['DATE'])
    new_pd['FILE_INDEX'] = file_index
    file_index = file_index + 1
    dfs.append(new_pd)
# Concatenate all data into one DataFrame
LTBX_FILM_RATINGS_DF = pd.concat(dfs, ignore_index=True).drop('Unnamed: 0',axis=1)
FILM_LIST = LTBX_FILM_RATINGS_DF.SHORT_URL.unique()

In [37]:
#Adding new column representing the number of new ratings
for short_url in FILM_LIST:
    results_index = LTBX_FILM_RATINGS_DF[LTBX_FILM_RATINGS_DF.SHORT_URL == short_url].index
    results_len = len(results_index)
    if results_len > 0:
        previous_position = LTBX_FILM_RATINGS_DF.loc[results_index[0]].SUM
        previous_appearance = LTBX_FILM_RATINGS_DF.loc[results_index[0]].FILE_INDEX
        is_first_appearance = True
        for index in results_index:
            if is_first_appearance: #All POP_DIFF will be 0
                LTBX_FILM_RATINGS_DF.loc[index, 'SUM_DIFF'] = 'NEW'
                is_first_appearance = False
            else: #Following entries being read
                current_appearance = LTBX_FILM_RATINGS_DF.loc[index].FILE_INDEX
                current_position = LTBX_FILM_RATINGS_DF.loc[index].SUM
                days_since_last_appearance = current_appearance - previous_appearance
                #if delta_appearance > 0: print(LTBX_FILMS_POP_DF.loc[index])
                if days_since_last_appearance == 1: #The movie was in the list the day before
                    LTBX_FILM_RATINGS_DF.loc[index, 'SUM_DIFF'] = current_position - previous_position
                else: #Filme saiu da lista e reentrou
                    LTBX_FILM_RATINGS_DF.loc[index, 'SUM_DIFF'] = 'OLD'
                previous_appearance = current_appearance
                previous_position = current_position    

In [38]:
LTBX_FILM_RATINGS_DF

,SHORT_URL,FANS,RATINGS,SUM,MEAN,DISCORD,DATE,N_FANS,MEAN_IND,DISCORD_IND,FILE_INDEX,SUM_DIFF
0,parasite-2019,54K fans,"[1505, 2681, 1277, 7410, 6531, 44606, 54854, 2...",1560371,4.576648,0.270837,2022-12-25 22:54:42.590243,54000,4577,271,0,NEW
1,joker-2019,14K fans,"[7319, 17083, 12006, 56067, 49965, 191778, 177...",1453337,3.888939,0.411148,2022-12-25 22:54:48.824093,14000,3889,411,0,NEW
2,fight-club,61K fans,"[1940, 4887, 2505, 16003, 13431, 86481, 93257,...",1251470,4.295418,0.334509,2022-12-25 22:54:55.060380,61000,4295,335,0,NEW
3,the-batman,26K fans,"[2521, 6506, 4471, 24511, 26265, 107699, 13838...",1147960,4.064136,0.355788,2022-12-25 22:55:01.317132,26000,4064,356,0,NEW
4,knives-out-2019,21K fans,"[1553, 5118, 3044, 21487, 21413, 130459, 15432...",1259348,4.054403,0.334665,2022-12-25 22:55:07.543282,21000,4054,335,0,NEW
...,...,...,...,...,...,...,...,...,...,...,...,...
110222,burlesque,948 fans,"[613, 1759, 1976, 5961, 6256, 14356, 9431, 108...",61594,3.293097,0.467404,2023-03-10 02:43:24.939098,948,3293,467,74,NEW
110223,catherine-called-birdy,62 fans,"[44, 123, 160, 547, 934, 3649, 4446, 4556, 904...",16259,3.511532,0.329538,2023-03-10 02:43:29.217258,62,3512,330,74,NEW
110224,65,7 fans,"[45, 161, 238, 509, 457, 579, 277, 140, 25, 45]",2476,2.546446,0.401453,2023-03-10 02:43:35.476526,7,2546,401,74,NEW
110225,ripd,11 fans,"[3579, 7829, 7064, 12268, 8497, 9052, 3251, 19...",54497,2.150219,0.429081,2023-03-10 02:43:41.740881,11,2150,429,74,NEW


In [39]:
LTBX_FILM_RATINGS_DF.to_csv(DF_LTBX_DIR + '\\LTBX_ALL_FILM_RATINGS_FROM_20221225_TO_' + datetime.today().strftime("%Y%m%d") + '.csv')

## 3. Films Watches and Likes (Numbers)

### 3.1 Scraping Most Current Film Numbers (Watches and Likes)
#### Retrieves ratings data from a cumulative film list

In [40]:
NEW_FILM_NUMBERS_DF = scrape_letterboxd_numbers(FILM_LIST[:])

{'SHORT_URL': 'parasite-2019', 'TITLE': 'Parasite', 'RELEASE_YEAR': 2019, 'TOTAL_AUDIENCE': 2399789, 'FANS': 60220, 'LIKES': 1385692, 'REVIEWS': 213432, 'LISTS': 384918, 'DATE': datetime.datetime(2023, 3, 10, 2, 47, 11, 617213)}
... waiting 3 seconds ... 1636 films pending
{'SHORT_URL': 'joker-2019', 'TITLE': 'Joker', 'RELEASE_YEAR': 2019, 'TOTAL_AUDIENCE': 2369277, 'FANS': 15279, 'LIKES': 972170, 'REVIEWS': 159072, 'LISTS': 221465, 'DATE': datetime.datetime(2023, 3, 10, 2, 47, 15, 35217)}
... waiting 4 seconds ... 1635 films pending
{'SHORT_URL': 'fight-club', 'TITLE': 'Fight Club', 'RELEASE_YEAR': 1999, 'TOTAL_AUDIENCE': 2204686, 'FANS': 68049, 'LIKES': 964529, 'REVIEWS': 143635, 'LISTS': 269541, 'DATE': datetime.datetime(2023, 3, 10, 2, 47, 19, 387653)}
... waiting 3 seconds ... 1634 films pending
{'SHORT_URL': 'the-batman', 'TITLE': 'The Batman', 'RELEASE_YEAR': 2022, 'TOTAL_AUDIENCE': 1709404, 'FANS': 30028, 'LIKES': 745296, 'REVIEWS': 365661, 'LISTS': 296354, 'DATE': datetime.dat

{'SHORT_URL': 'black-panther', 'TITLE': 'Black Panther', 'RELEASE_YEAR': 2018, 'TOTAL_AUDIENCE': 1935672, 'FANS': 3440, 'LIKES': 644272, 'REVIEWS': 76237, 'LISTS': 237385, 'DATE': datetime.datetime(2023, 3, 10, 2, 49, 19, 831050)}
... waiting 3 seconds ... 1606 films pending
{'SHORT_URL': 'baby-driver', 'TITLE': 'Baby Driver', 'RELEASE_YEAR': 2017, 'TOTAL_AUDIENCE': 1691258, 'FANS': 17687, 'LIKES': 609144, 'REVIEWS': 88441, 'LISTS': 159469, 'DATE': datetime.datetime(2023, 3, 10, 2, 49, 23, 203609)}
... waiting 5 seconds ... 1605 films pending
{'SHORT_URL': 'the-social-network', 'TITLE': 'The Social Network', 'RELEASE_YEAR': 2010, 'TOTAL_AUDIENCE': 1657650, 'FANS': 23066, 'LIKES': 521355, 'REVIEWS': 112247, 'LISTS': 204410, 'DATE': datetime.datetime(2023, 3, 10, 2, 49, 28, 574054)}
... waiting 5 seconds ... 1604 films pending
{'SHORT_URL': 'her', 'TITLE': 'Her', 'RELEASE_YEAR': 2013, 'TOTAL_AUDIENCE': 1377592, 'FANS': 25289, 'LIKES': 521535, 'REVIEWS': 92242, 'LISTS': 192267, 'DATE': da

{'SHORT_URL': 'doctor-strange-in-the-multiverse-of-madness', 'TITLE': 'Doctor Strange in the Multiverse of Madness', 'RELEASE_YEAR': 2022, 'TOTAL_AUDIENCE': 1048135, 'FANS': 1210, 'LIKES': 250704, 'REVIEWS': 218092, 'LISTS': 207609, 'DATE': datetime.datetime(2023, 3, 10, 2, 51, 26, 993704)}
... waiting 5 seconds ... 1576 films pending
{'SHORT_URL': 'blade-runner-2049', 'TITLE': 'Blade Runner 2049', 'RELEASE_YEAR': 2017, 'TOTAL_AUDIENCE': 1108590, 'FANS': 29872, 'LIKES': 422134, 'REVIEWS': 96612, 'LISTS': 204024, 'DATE': datetime.datetime(2023, 3, 10, 2, 51, 32, 429321)}
... waiting 4 seconds ... 1575 films pending
{'SHORT_URL': 'soul-2020', 'TITLE': 'Soul', 'RELEASE_YEAR': 2020, 'TOTAL_AUDIENCE': 1209480, 'FANS': 3593, 'LIKES': 429759, 'REVIEWS': 109776, 'LISTS': 152057, 'DATE': datetime.datetime(2023, 3, 10, 2, 51, 36, 779011)}
... waiting 5 seconds ... 1574 films pending
{'SHORT_URL': 'coco-2017', 'TITLE': 'Coco', 'RELEASE_YEAR': 2017, 'TOTAL_AUDIENCE': 1520223, 'FANS': 6782, 'LIKES'

{'SHORT_URL': 'captain-america-civil-war', 'TITLE': 'Captain America: Civil War', 'RELEASE_YEAR': 2016, 'TOTAL_AUDIENCE': 1504959, 'FANS': 2420, 'LIKES': 384438, 'REVIEWS': 51553, 'LISTS': 207300, 'DATE': datetime.datetime(2023, 3, 10, 2, 53, 46, 341238)}
... waiting 3 seconds ... 1546 films pending
{'SHORT_URL': 'no-country-for-old-men', 'TITLE': 'No Country for Old Men', 'RELEASE_YEAR': 2007, 'TOTAL_AUDIENCE': 1061563, 'FANS': 17107, 'LIKES': 381123, 'REVIEWS': 57691, 'LISTS': 180404, 'DATE': datetime.datetime(2023, 3, 10, 2, 53, 49, 712946)}
... waiting 4 seconds ... 1545 films pending
{'SHORT_URL': 'star-wars-the-force-awakens', 'TITLE': 'Star Wars: The Force Awakens', 'RELEASE_YEAR': 2015, 'TOTAL_AUDIENCE': 1490109, 'FANS': 1591, 'LIKES': 374646, 'REVIEWS': 56751, 'LISTS': 164652, 'DATE': datetime.datetime(2023, 3, 10, 2, 53, 54, 90817)}
... waiting 4 seconds ... 1544 films pending
{'SHORT_URL': '2001-a-space-odyssey', 'TITLE': '2001: A Space Odyssey', 'RELEASE_YEAR': 1968, 'TOTAL

{'SHORT_URL': 'alien', 'TITLE': 'Alien', 'RELEASE_YEAR': 1979, 'TOTAL_AUDIENCE': 992081, 'FANS': 20841, 'LIKES': 353173, 'REVIEWS': 59072, 'LISTS': 205996, 'DATE': datetime.datetime(2023, 3, 10, 2, 55, 54, 328851)}
... waiting 3 seconds ... 1517 films pending
{'SHORT_URL': 'the-dark-knight-rises', 'TITLE': 'The Dark Knight Rises', 'RELEASE_YEAR': 2012, 'TOTAL_AUDIENCE': 1341974, 'FANS': 5212, 'LIKES': 355054, 'REVIEWS': 45001, 'LISTS': 158982, 'DATE': datetime.datetime(2023, 3, 10, 2, 55, 57, 699642)}
... waiting 4 seconds ... 1516 films pending
{'SHORT_URL': 'ex-machina-2015', 'TITLE': 'Ex Machina', 'RELEASE_YEAR': 2014, 'TOTAL_AUDIENCE': 985897, 'FANS': 5510, 'LIKES': 322198, 'REVIEWS': 55478, 'LISTS': 135485, 'DATE': datetime.datetime(2023, 3, 10, 2, 56, 2, 79033)}
... waiting 5 seconds ... 1515 films pending
{'SHORT_URL': 'star-wars-the-last-jedi', 'TITLE': 'Star Wars: The Last Jedi', 'RELEASE_YEAR': 2017, 'TOTAL_AUDIENCE': 1254307, 'FANS': 2506, 'LIKES': 261741, 'REVIEWS': 75832, 

...

{'SHORT_URL': 'tampopo', 'TITLE': 'Tampopo', 'RELEASE_YEAR': 1985, 'TOTAL_AUDIENCE': 78763, 'FANS': 2693, 'LIKES': 33184, 'REVIEWS': 16138, 'LISTS': 35202, 'DATE': datetime.datetime(2023, 3, 10, 4, 37, 38, 571633)}
... waiting 5 seconds ... 132 films pending
{'SHORT_URL': 'pocahontas', 'TITLE': 'Pocahontas', 'RELEASE_YEAR': 1995, 'TOTAL_AUDIENCE': 512951, 'FANS': 222, 'LIKES': 54343, 'REVIEWS': 7762, 'LISTS': 35460, 'DATE': datetime.datetime(2023, 3, 10, 4, 37, 43, 956283)}
... waiting 3 seconds ... 131 films pending
{'SHORT_URL': 'the-founder', 'TITLE': 'The Founder', 'RELEASE_YEAR': 2016, 'TOTAL_AUDIENCE': 244119, 'FANS': 151, 'LIKES': 40424, 'REVIEWS': 15642, 'LISTS': 19641, 'DATE': datetime.datetime(2023, 3, 10, 4, 37, 47, 313855)}
... waiting 4 seconds ... 130 films pending
{'SHORT_URL': 'magic-mikes-last-dance', 'TITLE': "Magic Mike's Last Dance", 'RELEASE_YEAR': 2023, 'TOTAL_AUDIENCE': 24074, 'FANS': 37, 'LIKES': 4604, 'REVIEWS': 11155, 'LISTS': 7729, 'DATE': datetime.datetime(2

{'SHORT_URL': 'my-bloody-valentine', 'TITLE': 'My Bloody Valentine', 'RELEASE_YEAR': 1981, 'TOTAL_AUDIENCE': 50216, 'FANS': 215, 'LIKES': 13201, 'REVIEWS': 9451, 'LISTS': 19308, 'DATE': datetime.datetime(2023, 3, 10, 4, 39, 54, 707051)}
... waiting 5 seconds ... 102 films pending
{'SHORT_URL': 'the-flash', 'TITLE': 'The Flash', 'RELEASE_YEAR': 2023, 'TOTAL_AUDIENCE': 3287, 'FANS': 17, 'LIKES': 485, 'REVIEWS': 0, 'LISTS': 10111, 'DATE': datetime.datetime(2023, 3, 10, 4, 40, 0, 94021)}
... waiting 3 seconds ... 101 films pending
{'SHORT_URL': 'winnie-the-pooh-blood-and-honey', 'TITLE': 'Winnie the Pooh: Blood and Honey', 'RELEASE_YEAR': 2023, 'TOTAL_AUDIENCE': 9688, 'FANS': 13, 'LIKES': 1352, 'REVIEWS': 4839, 'LISTS': 4967, 'DATE': datetime.datetime(2023, 3, 10, 4, 40, 3, 487020)}
... waiting 4 seconds ... 100 films pending
{'SHORT_URL': 'my-bloody-valentine-2009', 'TITLE': 'My Bloody Valentine', 'RELEASE_YEAR': 2009, 'TOTAL_AUDIENCE': 46208, 'FANS': 46, 'LIKES': 5100, 'REVIEWS': 4375, '

{'SHORT_URL': 'uptown-girls', 'TITLE': 'Uptown Girls', 'RELEASE_YEAR': 2003, 'TOTAL_AUDIENCE': 84484, 'FANS': 1171, 'LIKES': 21210, 'REVIEWS': 8004, 'LISTS': 14309, 'DATE': datetime.datetime(2023, 3, 10, 4, 42, 5, 852222)}
... waiting 5 seconds ... 72 films pending
{'SHORT_URL': '47-meters-down-uncaged', 'TITLE': '47 Meters Down: Uncaged', 'RELEASE_YEAR': 2019, 'TOTAL_AUDIENCE': 49116, 'FANS': 22, 'LIKES': 4332, 'REVIEWS': 7804, 'LISTS': 9075, 'DATE': datetime.datetime(2023, 3, 10, 4, 42, 11, 270308)}
... waiting 5 seconds ... 71 films pending
{'SHORT_URL': 'huesera-the-bone-woman', 'TITLE': 'Huesera: The Bone Woman', 'RELEASE_YEAR': 2022, 'TOTAL_AUDIENCE': 9935, 'FANS': 22, 'LIKES': 3294, 'REVIEWS': 2741, 'LISTS': 3620, 'DATE': datetime.datetime(2023, 3, 10, 4, 42, 16, 679950)}
... waiting 3 seconds ... 70 films pending
{'SHORT_URL': 'pitch-perfect-2', 'TITLE': 'Pitch Perfect 2', 'RELEASE_YEAR': 2015, 'TOTAL_AUDIENCE': 353939, 'FANS': 564, 'LIKES': 39159, 'REVIEWS': 14099, 'LISTS': 23

{'SHORT_URL': 'platoon', 'TITLE': 'Platoon', 'RELEASE_YEAR': 1986, 'TOTAL_AUDIENCE': 222126, 'FANS': 579, 'LIKES': 47776, 'REVIEWS': 10443, 'LISTS': 51663, 'DATE': datetime.datetime(2023, 3, 10, 4, 44, 13, 978354)}
... waiting 4 seconds ... 42 films pending
{'SHORT_URL': 'champions-2023', 'TITLE': 'Champions', 'RELEASE_YEAR': 2023, 'TOTAL_AUDIENCE': 2171, 'FANS': 7, 'LIKES': 524, 'REVIEWS': 1003, 'LISTS': 1440, 'DATE': datetime.datetime(2023, 3, 10, 4, 44, 18, 395438)}
... waiting 4 seconds ... 41 films pending
{'SHORT_URL': 'whats-love-got-to-do-with-it-2023', 'TITLE': "What's Love Got to Do With It?", 'RELEASE_YEAR': 2022, 'TOTAL_AUDIENCE': 5212, 'FANS': 2, 'LIKES': 754, 'REVIEWS': 1551, 'LISTS': 1752, 'DATE': datetime.datetime(2023, 3, 10, 4, 44, 22, 769346)}
... waiting 5 seconds ... 40 films pending
{'SHORT_URL': 'nocebo-2022', 'TITLE': 'Nocebo', 'RELEASE_YEAR': 2022, 'TOTAL_AUDIENCE': 8415, 'FANS': 8, 'LIKES': 1723, 'REVIEWS': 2211, 'LISTS': 2186, 'DATE': datetime.datetime(2023, 

{'SHORT_URL': 'the-hangover-part-iii', 'TITLE': 'The Hangover Part III', 'RELEASE_YEAR': 2013, 'TOTAL_AUDIENCE': 294415, 'FANS': 47, 'LIKES': 24962, 'REVIEWS': 7104, 'LISTS': 15587, 'DATE': datetime.datetime(2023, 3, 10, 4, 46, 34, 324656)}
... waiting 3 seconds ... 11 films pending
{'SHORT_URL': 'history-of-the-world-part-i', 'TITLE': 'History of the World: Part I', 'RELEASE_YEAR': 1981, 'TOTAL_AUDIENCE': 39722, 'FANS': 80, 'LIKES': 7662, 'REVIEWS': 2507, 'LISTS': 7276, 'DATE': datetime.datetime(2023, 3, 10, 4, 46, 37, 754148)}
... waiting 5 seconds ... 10 films pending
{'SHORT_URL': 'sun-and-concrete', 'TITLE': 'Sun and Concrete', 'RELEASE_YEAR': 2023, 'TOTAL_AUDIENCE': 2054, 'FANS': 5, 'LIKES': 622, 'REVIEWS': 350, 'LISTS': 398, 'DATE': datetime.datetime(2023, 3, 10, 4, 46, 43, 170606)}
... waiting 3 seconds ... 9 films pending
{'SHORT_URL': 'spirit-stallion-of-the-cimarron', 'TITLE': 'Spirit: Stallion of the Cimarron', 'RELEASE_YEAR': 2002, 'TOTAL_AUDIENCE': 191559, 'FANS': 2071, '

In [41]:
len(FILM_LIST[:])

1637

In [42]:
NEW_FILM_NUMBERS_DF

,SHORT_URL,TITLE,RELEASE_YEAR,TOTAL_AUDIENCE,FANS,LIKES,REVIEWS,LISTS,DATE
0,parasite-2019,Parasite,2019,2399789,60220,1385692,213432,384918,2023-03-10 02:47:11.617213
1,joker-2019,Joker,2019,2369277,15279,972170,159072,221465,2023-03-10 02:47:15.035217
2,fight-club,Fight Club,1999,2204686,68049,964529,143635,269541,2023-03-10 02:47:19.387653
3,the-batman,The Batman,2022,1709404,30028,745296,365661,296354,2023-03-10 02:47:22.774866
4,knives-out-2019,Knives Out,2019,2099856,25997,949684,222938,234655,2023-03-10 02:47:26.127216
...,...,...,...,...,...,...,...,...,...
1632,burlesque,Burlesque,2010,122471,948,22897,10385,16088,2023-03-10 04:47:05.235901
1633,catherine-called-birdy,Catherine Called Birdy,2022,20641,62,5479,5951,6383,2023-03-10 04:47:10.643781
1634,65,65,2023,3139,7,510,1387,5392,2023-03-10 04:47:14.019832
1635,ripd,R.I.P.D.,2013,99213,11,6742,5071,7615,2023-03-10 04:47:18.468110


In [43]:
NEW_FILM_NUMBERS_DF.sort_values(by=['RELEASE_YEAR'], ascending = True).head(20)

,SHORT_URL,TITLE,RELEASE_YEAR,TOTAL_AUDIENCE,FANS,LIKES,REVIEWS,LISTS,DATE
1267,a-knives-out-mystery-3,,0,0,0,0,0,0,2023-03-10 04:20:00.101251
1338,killers-of-the-flower-moon,Killers of the Flower Moon,0,664,11,355,0,11745,2023-03-10 04:25:12.119552
1384,a-trip-to-the-moon,A Trip to the Moon,1902,273800,247,57727,20140,32662,2023-03-10 04:28:33.739605
1485,the-cabinet-of-dr-caligari-1920,The Cabinet of Dr. Caligari,1920,172447,1166,48163,16764,59970,2023-03-10 04:36:10.884525
885,nosferatu,Nosferatu,1922,210697,778,53418,20253,71281,2023-03-10 03:52:16.696452
786,metropolis,Metropolis,1927,212080,2000,58979,13118,70227,2023-03-10 03:44:59.245874
1200,the-passion-of-joan-of-arc,The Passion of Joan of Arc,1928,99761,1589,32377,12767,57110,2023-03-10 04:15:06.897443
1201,man-with-a-movie-camera,Man with a Movie Camera,1929,81024,665,24049,9299,36133,2023-03-10 04:15:11.311084
1247,city-lights,City Lights,1931,143449,1264,42398,10566,60371,2023-03-10 04:18:27.955864
1214,m,M,1931,161076,1298,48891,13966,68178,2023-03-10 04:16:08.488936


In [44]:
NEW_FILM_NUMBERS_DF.to_csv(DF_LTBX_DIR + '\\LTBX_FILM_NUMBERS_' + datetime.today().strftime("%Y%m%d") + '.csv')

### 3.2 Consolidating Daily Numbers Files

In [45]:
filenames = glob.glob(DF_LTBX_DIR + "\\LTBX_FILM_NUMBERS_*.csv")
dfs = []
file_index = 0
for filename in filenames:
    new_pd = pd.read_csv(filename, parse_dates=['DATE'])
    new_pd['FILE_INDEX'] = file_index
    file_index = file_index + 1
    dfs.append(new_pd)
# Concatenate all data into one DataFrame
LTBX_FILM_NUMBERS_DF = pd.concat(dfs, ignore_index=True).drop('Unnamed: 0',axis=1)
FILM_LIST = LTBX_FILM_NUMBERS_DF.SHORT_URL.unique()

In [46]:
filenames

['C:\\Users\\willi\\Documents\\Data Science\\Projeto Críticas\\Letterboxd\\Most Popular Films CSVs\\2023\\STAGE\\LTBX_FILM_NUMBERS_20221225.csv',
 'C:\\Users\\willi\\Documents\\Data Science\\Projeto Críticas\\Letterboxd\\Most Popular Films CSVs\\2023\\STAGE\\LTBX_FILM_NUMBERS_20221226.csv',
 'C:\\Users\\willi\\Documents\\Data Science\\Projeto Críticas\\Letterboxd\\Most Popular Films CSVs\\2023\\STAGE\\LTBX_FILM_NUMBERS_20221227.csv',
 'C:\\Users\\willi\\Documents\\Data Science\\Projeto Críticas\\Letterboxd\\Most Popular Films CSVs\\2023\\STAGE\\LTBX_FILM_NUMBERS_20221228.csv',
 'C:\\Users\\willi\\Documents\\Data Science\\Projeto Críticas\\Letterboxd\\Most Popular Films CSVs\\2023\\STAGE\\LTBX_FILM_NUMBERS_20221229.csv',
 'C:\\Users\\willi\\Documents\\Data Science\\Projeto Críticas\\Letterboxd\\Most Popular Films CSVs\\2023\\STAGE\\LTBX_FILM_NUMBERS_20221230.csv',
 'C:\\Users\\willi\\Documents\\Data Science\\Projeto Críticas\\Letterboxd\\Most Popular Films CSVs\\2023\\STAGE\\LTBX_FILM_N

In [47]:
#Adding new column representing the daily difference of numbers
for short_url in FILM_LIST:
    results_index = LTBX_FILM_NUMBERS_DF[LTBX_FILM_NUMBERS_DF.SHORT_URL == short_url].index
    results_len = len(results_index)
    if results_len > 0:
        previous_audience = LTBX_FILM_NUMBERS_DF.loc[results_index[0]].TOTAL_AUDIENCE
        previous_fans = LTBX_FILM_NUMBERS_DF.loc[results_index[0]].FANS
        previous_likes = LTBX_FILM_NUMBERS_DF.loc[results_index[0]].LIKES
        previous_reviews = LTBX_FILM_NUMBERS_DF.loc[results_index[0]].REVIEWS
        previous_appearance = LTBX_FILM_NUMBERS_DF.loc[results_index[0]].FILE_INDEX
        is_first_appearance = True
        for index in results_index:
            if is_first_appearance: #All AUDIENCE_DIFF will be 0 ('NEW')
                LTBX_FILM_NUMBERS_DF.loc[index, 'AUDIENCE_DIFF'] = 'NEW'
                LTBX_FILM_NUMBERS_DF.loc[index, 'FANS_DIFF'] = 'NEW'
                LTBX_FILM_NUMBERS_DF.loc[index, 'LIKES_DIFF'] = 'NEW'
                LTBX_FILM_NUMBERS_DF.loc[index, 'REVIEWS_DIFF'] = 'NEW'
                is_first_appearance = False
            else: #Following entries being read
                current_appearance = LTBX_FILM_NUMBERS_DF.loc[index].FILE_INDEX
                current_audience = LTBX_FILM_NUMBERS_DF.loc[index].TOTAL_AUDIENCE
                current_fans = LTBX_FILM_NUMBERS_DF.loc[index].FANS
                current_likes = LTBX_FILM_NUMBERS_DF.loc[index].LIKES
                current_reviews = LTBX_FILM_NUMBERS_DF.loc[index].REVIEWS
                days_since_last_appearance = current_appearance - previous_appearance
                if days_since_last_appearance == 1: #The movie was in the list the day before
                    LTBX_FILM_NUMBERS_DF.loc[index, 'AUDIENCE_DIFF'] = current_audience - previous_audience
                    LTBX_FILM_NUMBERS_DF.loc[index, 'FANS_DIFF'] = current_fans - previous_fans
                    LTBX_FILM_NUMBERS_DF.loc[index, 'LIKES_DIFF'] = current_likes - previous_likes
                    LTBX_FILM_NUMBERS_DF.loc[index, 'REVIEWS_DIFF'] = current_reviews - previous_reviews
                else: #Filme saiu da lista e reentrou
                    LTBX_FILM_NUMBERS_DF.loc[index, 'AUDIENCE_DIFF'] = 'OLD'
                    LTBX_FILM_NUMBERS_DF.loc[index, 'FANS_DIFF'] = 'OLD'
                    LTBX_FILM_NUMBERS_DF.loc[index, 'LIKES_DIFF'] = 'OLD'
                    LTBX_FILM_NUMBERS_DF.loc[index, 'REVIEWS_DIFF'] = 'OLD'
                previous_appearance = current_appearance
                previous_audience = current_audience   
                previous_fans = current_fans
                previous_likes = current_likes
                previous_reviews = current_reviews

In [48]:
LTBX_FILM_NUMBERS_DF.to_csv(DF_LTBX_DIR + '\\LTBX_ALL_FILM_NUMBERS_FROM_20221225_TO_' + datetime.today().strftime("%Y%m%d") + '.csv')

-----------------------------------

In [49]:
FIX_DF = LTBX_FILM_RATINGS_DF[LTBX_FILM_RATINGS_DF.FILE_INDEX == 15]

In [50]:
LTBX_FILM_RATINGS_DF[LTBX_FILM_RATINGS_DF.SHORT_URL == 'sympathy-for-lady-vengeance']

,SHORT_URL,FANS,RATINGS,SUM,MEAN,DISCORD,DATE,N_FANS,MEAN_IND,DISCORD_IND,FILE_INDEX,SUM_DIFF
1244,sympathy-for-lady-vengeance,1.7K fans,"[96, 221, 269, 1035, 1693, 7275, 13486, 25570,...",71030,3.933458,0.315501,2022-12-26 01:24:52.244382,1700,3933,316,0,NEW
2548,sympathy-for-lady-vengeance,1.7K fans,"[96, 221, 268, 1035, 1694, 7273, 13495, 25579,...",71050,3.933505,0.315455,2022-12-26 12:38:21.171232,1700,3934,315,1,20
3859,sympathy-for-lady-vengeance,1.7K fans,"[96, 221, 268, 1037, 1698, 7276, 13512, 25612,...",71126,3.933491,0.315457,2022-12-27 13:00:27.273620,1700,3933,315,2,76
5155,sympathy-for-lady-vengeance,1.7K fans,"[96, 221, 268, 1038, 1699, 7282, 13527, 25654,...",71210,3.933514,0.315384,2022-12-28 13:20:14.759542,1700,3934,315,3,84
6471,sympathy-for-lady-vengeance,1.7K fans,"[96, 221, 268, 1040, 1704, 7294, 13555, 25693,...",71328,3.933406,0.315359,2022-12-29 21:44:18.595972,1700,3933,315,4,118
...,...,...,...,...,...,...,...,...,...,...,...,...
103321,sympathy-for-lady-vengeance,1.9K fans,"[103, 239, 280, 1103, 1838, 7898, 14727, 28166...",78150,3.940621,0.314606,2023-03-06 09:25:58.367490,1900,3941,315,70,124
104945,sympathy-for-lady-vengeance,1.9K fans,"[103, 239, 282, 1104, 1838, 7906, 14738, 28194...",78222,3.940624,0.314625,2023-03-07 09:20:41.556118,1900,3941,315,71,72
106571,sympathy-for-lady-vengeance,1.9K fans,"[103, 239, 283, 1107, 1842, 7911, 14757, 28212...",78298,3.940567,0.314686,2023-03-08 09:45:23.817464,1900,3941,315,72,76
108202,sympathy-for-lady-vengeance,1.9K fans,"[103, 239, 283, 1108, 1847, 7918, 14762, 28229...",78350,3.940511,0.314707,2023-03-09 01:03:05.843145,1900,3941,315,73,52


In [51]:
LTBX_FILM_NUMBERS_DF[LTBX_FILM_NUMBERS_DF.SHORT_URL == 'sympathy-for-lady-vengeance']

,SHORT_URL,TITLE,RELEASE_YEAR,TOTAL_AUDIENCE,FANS,LIKES,REVIEWS,LISTS,DATE,FILE_INDEX,AUDIENCE_DIFF,FANS_DIFF,LIKES_DIFF,REVIEWS_DIFF
1244,sympathy-for-lady-vengeance,Sympathy for Lady Vengeance,2005,107535,1709,32780,10895,33938,2022-12-29 01:50:49.649001,0,NEW,NEW,NEW,NEW
2560,sympathy-for-lady-vengeance,Sympathy for Lady Vengeance,2005,107535,1709,32780,10895,33938,2022-12-29 01:50:49.649001,1,0,0,0,0
3876,sympathy-for-lady-vengeance,Sympathy for Lady Vengeance,2005,107535,1709,32780,10895,33938,2022-12-29 01:50:49.649001,2,0,0,0,0
5192,sympathy-for-lady-vengeance,Sympathy for Lady Vengeance,2005,107535,1709,32780,10895,33938,2022-12-29 01:50:49.649001,3,0,0,0,0
6508,sympathy-for-lady-vengeance,Sympathy for Lady Vengeance,2005,107650,1713,32824,10912,34007,2022-12-30 01:18:34.709742,4,115,4,44,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103358,sympathy-for-lady-vengeance,NaN,2005,0,0,0,0,0,2023-03-06 11:36:36.761131,70,0,0,0,0
104982,sympathy-for-lady-vengeance,NaN,2005,0,0,0,0,0,2023-03-07 17:52:59.683881,71,0,0,0,0
106608,sympathy-for-lady-vengeance,NaN,2005,0,0,0,0,0,2023-03-08 11:53:57.421587,72,0,0,0,0
108239,sympathy-for-lady-vengeance,NaN,2005,0,0,0,0,0,2023-03-09 03:14:13.756107,73,0,0,0,0


In [52]:
LTBX_FILM_NUMBERS_DF[LTBX_FILM_NUMBERS_DF.SHORT_URL == 'lady-vengeance']

,SHORT_URL,TITLE,RELEASE_YEAR,TOTAL_AUDIENCE,FANS,LIKES,REVIEWS,LISTS,DATE,FILE_INDEX,AUDIENCE_DIFF,FANS_DIFF,LIKES_DIFF,REVIEWS_DIFF
25604,lady-vengeance,Lady Vengeance,2005,109856,1764,33585,11268,34959,2023-01-12 22:48:25.713670,18,NEW,NEW,NEW,NEW
26985,lady-vengeance,Lady Vengeance,2005,110009,1768,33660,11289,35000,2023-01-14 00:22:02.617739,19,153,4,75,21
28371,lady-vengeance,Lady Vengeance,2005,110210,1774,33724,11327,35048,2023-01-15 02:03:14.266427,20,201,6,64,38
29760,lady-vengeance,Lady Vengeance,2005,110421,1778,33801,11361,35081,2023-01-16 02:47:12.844358,21,211,4,77,34
31155,lady-vengeance,Lady Vengeance,2005,110563,1784,33847,11388,35138,2023-01-17 00:41:30.504635,22,142,6,46,27
32554,lady-vengeance,Lady Vengeance,2005,110698,1780,33892,11409,35173,2023-01-17 23:37:31.389659,23,135,-4,45,21
33956,lady-vengeance,Lady Vengeance,2005,110861,1785,33942,11433,35218,2023-01-19 00:11:37.987311,24,163,5,50,24
35361,lady-vengeance,Lady Vengeance,2005,110971,1784,33989,11451,35267,2023-01-19 23:51:56.154418,25,110,-1,47,18
36768,lady-vengeance,Lady Vengeance,2005,111098,1789,34037,11465,35308,2023-01-20 23:11:43.811339,26,127,5,48,14
38178,lady-vengeance,Lady Vengeance,2005,111247,1793,34094,11497,35348,2023-01-21 23:41:37.339598,27,149,4,57,32


In [53]:
filename = DF_LTBX_DIR + '\\LTBX_FILM_NUMBERS_20230118.csv'
FIX_DF = pd.read_csv(filename).drop('Unnamed: 0',axis=1)

In [54]:
FIX_DF[FIX_DF.SHORT_URL == 'sympathy-for-lady-vengeance']

,SHORT_URL,TITLE,RELEASE_YEAR,TOTAL_AUDIENCE,FANS,LIKES,REVIEWS,LISTS,DATE
1244,sympathy-for-lady-vengeance,Sympathy for Lady Vengeance,2005,109533,1755,33487,11214,34866,2023-01-19 00:01:58.528025


In [55]:
FIX_DF.loc[1244]

SHORT_URL         sympathy-for-lady-vengeance
TITLE             Sympathy for Lady Vengeance
RELEASE_YEAR                             2005
TOTAL_AUDIENCE                         109533
FANS                                     1755
LIKES                                   33487
REVIEWS                                 11214
LISTS                                   34866
DATE               2023-01-19 00:01:58.528025
Name: 1244, dtype: object

In [56]:
LTBX_FILM_NUMBERS_DF.loc[22726]

SHORT_URL         sympathy-for-lady-vengeance
TITLE             Sympathy for Lady Vengeance
RELEASE_YEAR                             2005
TOTAL_AUDIENCE                         109533
FANS                                     1755
LIKES                                   33487
REVIEWS                                 11214
LISTS                                   34866
DATE               2023-01-10 22:32:53.326604
FILE_INDEX                                 16
AUDIENCE_DIFF                             145
FANS_DIFF                                  -1
LIKES_DIFF                                 56
REVIEWS_DIFF                               20
Name: 22726, dtype: object

In [57]:
output_index = 427
input_index = 19136
FIX_DF.loc[output_index,'FANS'] = LTBX_FILM_RATINGS_DF.loc[input_index,'FANS']
FIX_DF.loc[output_index,'RATINGS'] = LTBX_FILM_RATINGS_DF.loc[input_index,'RATINGS']
FIX_DF.loc[output_index,'SUM'] = LTBX_FILM_RATINGS_DF.loc[input_index,'SUM']
FIX_DF.loc[output_index,'MEAN'] = LTBX_FILM_RATINGS_DF.loc[input_index,'MEAN']
FIX_DF.loc[output_index,'DISCORD'] = LTBX_FILM_RATINGS_DF.loc[input_index,'DISCORD']
FIX_DF.loc[output_index,'N_FANS'] = LTBX_FILM_RATINGS_DF.loc[input_index,'N_FANS']
FIX_DF.loc[output_index,'MEAN_IND'] = LTBX_FILM_RATINGS_DF.loc[input_index,'MEAN_IND']
FIX_DF.loc[output_index,'DISCORD_IND'] = LTBX_FILM_RATINGS_DF.loc[input_index,'DISCORD_IND']

In [58]:
output_index = 1244
input_index = 22726
FIX_DF.loc[output_index,'TITLE'] = LTBX_FILM_NUMBERS_DF.loc[input_index,'TITLE']
FIX_DF.loc[output_index,'RELEASE_YEAR'] = LTBX_FILM_NUMBERS_DF.loc[input_index,'RELEASE_YEAR']
FIX_DF.loc[output_index,'TOTAL_AUDIENCE'] = LTBX_FILM_NUMBERS_DF.loc[input_index,'TOTAL_AUDIENCE']
FIX_DF.loc[output_index,'FANS'] = LTBX_FILM_NUMBERS_DF.loc[input_index,'FANS']
FIX_DF.loc[output_index,'LIKES'] = LTBX_FILM_NUMBERS_DF.loc[input_index,'LIKES']
FIX_DF.loc[output_index,'REVIEWS'] = LTBX_FILM_NUMBERS_DF.loc[input_index,'REVIEWS']
FIX_DF.loc[output_index,'LISTS'] = LTBX_FILM_NUMBERS_DF.loc[input_index,'LISTS']

In [59]:
#FIX_DF.to_csv(filename)

In [60]:
FIX_DF[FIX_DF.SHORT_URL == 'crazy-rich-asians']

,SHORT_URL,TITLE,RELEASE_YEAR,TOTAL_AUDIENCE,FANS,LIKES,REVIEWS,LISTS,DATE,RATINGS,SUM,MEAN,DISCORD,N_FANS,MEAN_IND,DISCORD_IND
413,crazy-rich-asians,Crazy Rich Asians,2018,554193,2369,128207,43435,60080,2023-01-18 22:59:20.318094,NaN,NaN,NaN,NaN,NaN,NaN,NaN
